# Data Mining Project - Gene data

Sean Deloddere - 914691 <br>
Giulio Marcon - 

## Load in data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [2]:
ms_df = pd.read_csv('genedata.csv', index_col = 0)
ms_df.head()

,class,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f6991,f6992,f6993,f6994,f6995,f6996,f6997,f6998,f6999,f7000
id,,,,,,,,,,,,,,,,,,,,,
1,5,2.3383,10.5440,11.4653,10.4441,10.6311,7.7036,9.6444,7.8501,7.7853,...,11.9394,11.1349,5.7066,8.2176,9.9062,9.0815,10.5098,11.8648,8.4341,6.7618
2,4,3.9151,9.5815,10.3992,9.8333,8.9781,7.0265,9.2761,7.3903,8.2744,...,11.6547,7.7178,9.3558,7.5026,10.2106,8.9278,10.2466,11.3329,7.8209,7.4350
3,5,2.9322,9.3102,11.0756,9.8490,9.4291,8.0618,8.3874,7.9274,10.0544,...,11.9372,11.2369,6.5860,9.1668,10.3315,9.7551,11.4392,10.7036,8.0041,8.5380
4,5,3.9470,8.5315,10.9836,9.3676,9.6856,7.5842,8.5774,8.3737,9.0443,...,13.2541,11.8221,9.5261,7.4231,11.2862,9.8992,10.4067,11.9044,7.9843,8.5370
5,1,4.4618,9.1392,10.9183,10.0490,9.8290,6.5658,9.8122,9.0646,8.3383,...,11.6416,11.5142,6.5539,7.8319,11.0100,10.1013,11.2682,10.8969,8.1959,8.2906


In [3]:
ms_cols_with_missing = [col for col in ms_df.columns if ms_df[col].isnull().any()]

print("ms dataframe has "+str(len(ms_cols_with_missing))+" columns with missing values.")

ms dataframe has 0 columns with missing values.


In [4]:
if len(ms_df.columns) == 7001:
    true_labels = ms_df['class']
    ms_df = ms_df.drop('class',axis=1)

In [5]:
ms = ms_df.to_numpy()
ms = ms.astype(np.float)
print(ms)

[[ 2.3383 10.544  11.4653 ... 11.8648  8.4341  6.7618]
 [ 3.9151  9.5815 10.3992 ... 11.3329  7.8209  7.435 ]
 [ 2.9322  9.3102 11.0756 ... 10.7036  8.0041  8.538 ]
 ...
 [ 4.6489  9.0206 10.6455 ... 11.4399  8.3824  8.2929]
 [ 3.3839 10.6693 11.4684 ... 11.5674  8.3261  9.6444]
 [ 3.1952  9.5168 10.8056 ... 10.8684  8.3932  8.6758]]


## Pre-processing

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import DistanceMetric

In [7]:
ss = StandardScaler()
ms_ss = ss.fit_transform(ms)

In [8]:
mas = MaxAbsScaler()
ms_mas = mas.fit_transform(ms)

In [9]:
ms_cs = cosine_similarity(ms)
ms_cs_ss = cosine_similarity(ms_ss)
ms_cs_mas = cosine_similarity(ms_mas)

In [10]:
def euclidean_similarity(data):
    return 1/(1+euclidean_distances(data, data))

In [11]:
ms_ed = euclidean_similarity(ms)
ms_ed_ss = euclidean_similarity(ms_ss)
ms_ed_mas = euclidean_similarity(ms_mas)

## Clustering

In [12]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [13]:
def get_nmi(labels1,labels2):
    return normalized_mutual_info_score(labels1,labels2,average_method='geometric')

### K-Means

In [14]:
def perform_KMeans(data):
    kmeans_list = []
    for i in range(2,9):
        kmeans = KMeans(n_clusters=i).fit(data)
        labels = kmeans.labels_
        kmeans_list.append((i,get_nmi(true_labels,labels),labels))
    sorted_kmeans_list = sorted(kmeans_list, key=lambda x:x[1], reverse=True)
    return sorted_kmeans_list

In [15]:
kmeans_0 = perform_KMeans(ms)
kmeans_ss = perform_KMeans(ms_ss)
kmeans_mas = perform_KMeans(ms_mas)

In [16]:
for i in range(3):
    print(i)
    print(kmeans_0[i][0:2])
    print(kmeans_ss[i][0:2])
    print(kmeans_mas[i][0:2])

0
(6, 0.8828472279219505)
(5, 0.7983131078494884)
(5, 0.8457603627237771)
1
(5, 0.8569187368299778)
(8, 0.7820601648362181)
(7, 0.8441523317105245)
2
(7, 0.8563370586397301)
(7, 0.7740531891086693)
(6, 0.8255713528147901)


### Hierarchical

In [17]:
def perform_hierarchical(data,linkage):
    hier_list = []
    for i in range(2,9):
        hier = AgglomerativeClustering(n_clusters=i,linkage=linkage).fit(data)
        labels = hier.labels_
        hier_list.append((i,get_nmi(true_labels,labels),labels))
    sorted_hier_list = sorted(hier_list, key=lambda x:x[1], reverse=True)
    return sorted_hier_list        

In [18]:
linkages = ['ward','complete','average','single']
hier_0 = []
hier_ss = []
hier_mas = []
for linkage in linkages:
    print(linkage)
    hier_0.append((linkage,perform_hierarchical(ms,linkage)))
    hier_ss.append((linkage,perform_hierarchical(ms_ss,linkage)))
    hier_mas.append((linkage,perform_hierarchical(ms_mas,linkage)))

ward
complete
average
single


In [19]:
for i in range(4):
    print("============================================")
    print(linkages[i])
    print("0")
    for j in range(3):
        print(hier_0[i][1][j][0:2])
    print("ss")
    for j in range(3):
        print(hier_ss[i][1][j][0:2])
    print("mas")
    for j in range(3):
        print(hier_mas[i][1][j][0:2])

ward
0
(6, 0.9179157442581702)
(7, 0.8878777908498132)
(5, 0.8775855633609511)
ss
(7, 0.8378882021311446)
(8, 0.8282447174039829)
(6, 0.8169633520573191)
mas
(7, 0.88845314017232)
(6, 0.8512060961094837)
(5, 0.8480166279599085)
complete
0
(8, 0.6100507558366003)
(7, 0.6003261469427305)
(6, 0.5402558200309042)
ss
(8, 0.5083856029139342)
(7, 0.49403066238187526)
(6, 0.3202895588179002)
mas
(8, 0.6181491399834758)
(7, 0.5279598097586954)
(5, 0.30224608670436504)
average
0
(8, 0.046350697557658097)
(7, 0.04604423405299482)
(6, 0.040272133718214496)
ss
(8, 0.0765473397628901)
(7, 0.07449474820534495)
(6, 0.0378143980614158)
mas
(8, 0.0493062446937199)
(7, 0.04404218670429313)
(6, 0.040329542969028694)
single
0
(8, 0.03600217293124191)
(7, 0.03468494456958169)
(6, 0.03005573358925145)
ss
(8, 0.03600217293124191)
(7, 0.03155397027375823)
(6, 0.02995737119846597)
mas
(8, 0.03600217293124191)
(7, 0.031643766137050604)
(6, 0.03005573358925145)


### Spectral 

#### Nearest Neighbor

In [20]:
def perform_spectral_nn(data):
    spectral_list = []
    for nc in range(2,8):
        for nn in range(5,9,2):
            spectral = SpectralClustering(n_clusters=nc, affinity='nearest_neighbors',n_neighbors=nn,n_jobs=3).fit(data)
            labels = spectral.labels_
            spectral_list.append((nc,nn,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[2], reverse=True)
    return sorted_spectral_list 

In [21]:
spectral_0 = perform_spectral_nn(ms)
spectral_ss = perform_spectral_nn(ms_ss)
spectral_mas = perform_spectral_nn(ms_mas)

In [22]:
print("0")
for i in range(3):
    print(spectral_0[i][0:3])

print("ss")
for i in range(3):
    print(spectral_ss[i][0:3])

print("mas")
for i in range(3):
    print(spectral_mas[i][0:3])

0
(5, 5, 0.9847702095491678)
(5, 7, 0.9762785050842379)
(6, 5, 0.937244416777298)
ss
(5, 7, 0.9665000951941086)
(5, 5, 0.9485802550113134)
(6, 7, 0.9247786084922212)
mas
(5, 7, 0.9774353402417695)
(5, 5, 0.9759616118626921)
(6, 5, 0.9323883209786678)


#### RBF

In [23]:
def perform_spectral_rbf(data):
    spectral_list = []
    for nc in range(2,8):
        spectral = SpectralClustering(n_clusters=nc, affinity='rbf',n_jobs=3).fit(data)
        labels = spectral.labels_
        spectral_list.append((nc,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[1], reverse=True)
    return sorted_spectral_list 

In [24]:
spectral_rbf_0 = perform_spectral_rbf(ms)
spectral_rbf_ss = perform_spectral_rbf(ms_ss)
spectral_rbf_mas = perform_spectral_rbf(ms_mas)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\ProgramData\Anaconda3\lib

In [25]:
print("0")
for i in range(3):
    print(spectral_rbf_0[i][0:2])

print("ss")
for i in range(3):
    print(spectral_rbf_ss[i][0:2])

print("mas") 
for i in range(3):
    print(spectral_rbf_mas[i][0:2])

0
(7, 0.009210843866308958)
(6, 0.008758700113573834)
(5, 0.005220641092557486)
ss
(3, 0.00857213572828385)
(7, 0.00809520481052549)
(4, 0.00666139979900002)
mas
(7, 0.030949001742768982)
(6, 0.03005573358925145)
(5, 0.028462663673074918)


#### Precomputed

In [26]:
def perform_spectral_pc(data):
    spectral_list = []
    for nc in range(2,8):
        spectral = SpectralClustering(n_clusters=nc, affinity='precomputed',n_jobs=3).fit(data)
        labels = spectral.labels_
        spectral_list.append((nc,get_nmi(true_labels,labels),labels))
    sorted_spectral_list = sorted(spectral_list, key=lambda x:x[1], reverse=True)
    return sorted_spectral_list 

In [33]:
spectral_pc_cs = perform_spectral_pc(ms_cs)
#spectral_pc_cs_ss = perform_spectral_pc(ms_cs_ss)
spectral_pc_cs_mas = perform_spectral_pc(ms_cs_mas)
spectral_pc_ed = perform_spectral_pc(ms_ed)
spectral_pc_ed_ss = perform_spectral_pc(ms_ed_ss)
spectral_pc_ed_mas = perform_spectral_pc(ms_ed_mas)

In [35]:
print("cs 0")
for i in range(3):
    print(spectral_pc_cs[i][0:2])
#print(spectral_pc_cs_ss[0][0:2])
print("cs mas")
for i in range(3):
    print(spectral_pc_cs_mas[i][0:2])
print("ed 0")
for i in range(3):
    print(spectral_pc_ed[i][0:2])
print("ed ss")
for i in range(3):
    print(spectral_pc_ed_ss[i][0:2])
print("ed mas")
for i in range(3):
    print(spectral_pc_ed_mas[i][0:2])

cs 0
(6, 0.7780467277505074)
(4, 0.7543911337633787)
(7, 0.7532927018460793)
cs mas
(6, 0.7541808973568883)
(7, 0.730176447667632)
(5, 0.7131529597979378)
ed 0
(5, 0.8058333037259116)
(6, 0.7327814681446791)
(7, 0.7264881394798037)
ed ss
(7, 0.662545887933534)
(3, 0.651418154812908)
(6, 0.6189748931899598)
ed mas
(5, 0.7588423453942449)
(6, 0.7119441548736841)
(4, 0.7006921375277986)


# BEST METHOD

No Preprocessing  + Spectral Clustering + Nearest Neighbors (5 clusters, 5 neighbors)